# Digit Recognizer using multi-layer CNN
* https://www.kaggle.com/c/digit-recognizer

In [1]:
import numpy as np
import pandas as pd

## prepareing data
Download data from https://www.kaggle.com/c/digit-recognizer/data

In [2]:
n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)

validation_size = 2000

In [3]:
train = pd.read_csv('/home/carnd/datasets/digit-recognizer/train.csv')
test  = pd.read_csv('/home/carnd/datasets/digit-recognizer/test.csv')

In [4]:
(train.shape, test.shape)

((42000, 785), (28000, 784))

split train data to labels and pixels.

In [5]:
features = (train.ix[:,1:].values).astype('float32')
labels = pd.get_dummies(train.ix[:,0]).astype('float32')
(features.shape, labels.shape)

((42000, 784), (42000, 10))

In [6]:
# split data into training & validation
valid_features = features[:validation_size]
valid_labels = labels[:validation_size]

train_features = features[validation_size:]
train_labels = labels[validation_size:]
(train_features.shape, train_labels.shape, valid_features.shape, valid_labels.shape)

((40000, 784), (40000, 10), (2000, 784), (2000, 10))

In [7]:
test_features = (test.values).astype('float32')
(test_features.shape)

(28000, 784)

In [8]:
features = features.reshape(-1,28,28)
valid_features = valid_features.reshape(-1,28,28)
test_features = test_features.reshape(-1, 28, 28)

(features.shape, valid_features.shape, test_features.shape)

((42000, 28, 28), (2000, 28, 28), (28000, 28, 28))

In [9]:
features = np.expand_dims(features,3)
valid_features = np.expand_dims(valid_features,3)
test_features = np.expand_dims(test_features,3)
(features.shape, valid_features.shape, test_features.shape)

((42000, 28, 28, 1), (2000, 28, 28, 1), (28000, 28, 28, 1))

In [10]:
mean_px = features.mean().astype(np.float32)
std_px = features.std().astype(np.float32)

In [11]:
def norm_input(x): return (x-mean_px)/std_px

## Make a TensorFlow Graph

Make a first convolultion layer.

In [12]:
import tensorflow as tf
from tensorflow.contrib.keras.python.keras.models import Sequential
from tensorflow.contrib.keras.python.keras.layers import Flatten, Dense, Dropout, Lambda, Conv2D, BatchNormalization, MaxPool2D
from tensorflow.contrib.keras.python.keras.optimizers import Adam
from tensorflow.contrib.keras.python.keras.preprocessing.image import ImageDataGenerator

In [14]:
def get_model_bn_do():
    model = Sequential()
    model.add(Lambda(norm_input, input_shape=(28,28,1)))
    model.add(Conv2D(filters=32,kernel_size=(3,3), activation='relu'))
    model.add(BatchNormalization(axis=1))
    model.add(Conv2D(filters=32,kernel_size=(3,3), activation='relu'))
    model.add(MaxPool2D())
    model.add(BatchNormalization(axis=1))
    model.add(Conv2D(filters=64,kernel_size=(3,3), activation='relu'))
    model.add(BatchNormalization(axis=1))
    model.add(Conv2D(filters=64,kernel_size=(3,3), activation='relu'))
    model.add(MaxPool2D())
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [15]:
model = get_model_bn_do()

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 26, 26, 32)        104       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 12, 12, 32)        48        
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 64)        18496     
__________

## Training

In [17]:
batch_size = 64

In [39]:
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                            height_shift_range=0.08, zoom_range=0.08)
batches = gen.flow(features, labels, batch_size=batch_size)
test_batches = gen.flow(valid_features, valid_labels, batch_size=batch_size)

In [40]:
model.fit_generator(batches, batches.n//batch_size, epochs=1,
                    validation_data=test_batches, validation_steps=test_batches.n//batch_size)

Epoch 1/1
656/656 [==============================] - 268s - loss: 0.1443 - acc: 0.9565 - val_loss: 0.0481 - val_acc: 0.9829


## Testing

In [41]:
classes = model.predict_classes(test_features)

28000/28000 [==============================] - 110s   


In [42]:
classes[:5]

array([2, 0, 9, 9, 3])

## Write to file

In [43]:
submissions = pd.DataFrame({"ImageId": list(range(1, len(classes)+1)),
                             "Label": classes})
submissions.to_csv("output.csv", index=False, header=True)